# Prediction

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import json
from ipywidgets import ToggleButton, Button, VBox, HBox, IntSlider, interactive_output, Label, Dropdown, Output, ToggleButtons
from IPython.display import display, clear_output
from typing import Optional, Dict

# === Constants ===
CPU_COLOR = "#17a2b8"  # Bootstrap "info" blue
NIC_COLOR = "#28a745"  # Bootstrap "success" green
CPU_MODEL_FOLDER = "data/cpu_models"
MODEL_OUTPUT = "cpu_model"

# === NIC Power Database ===
NIC_POWER_DATABASE = {
    "Intel 10-Gigabit X540-AT2": 7.5,
    "82579LM Gigabit": 4.0,
    "Intel E810-XXV": 12.5
}

def nic_power_lookup(model_name: str) -> float:
    return NIC_POWER_DATABASE.get(model_name, 0.0)

def interactive_prediction_plots(cpu_model: Optional[Dict], method: str = "linear") -> None:
    if cpu_model is None:
        raise ValueError("cpu_model is required.")

    num_cores = cpu_model.get("cpu_info", {}).get("cores", 4)

    load_options = {
        "Idle (0%)": 0.0,
        "25% Load": 0.25,
        "50% Load": 0.5,
        "75% Load": 0.75,
        "100% Load": 1.0
    }

    load_buttons = {
        name: ToggleButton(
            value=True,
            description=name,
            tooltip=f"Simulate server power consumption under {name} CPU usage",
            layout={'width': '150px'},
            button_style='info'
        )
        for name in load_options.keys()
    }

    nic_models = list(NIC_POWER_DATABASE.keys())
    nic_buttons = {
        model: ToggleButton(
            value=True,
            description=model,
            tooltip=model,
            layout={'width': '140px'},
            button_style='success'
        )
        for model in nic_models
    }

    def update_button_style(button, active_style: str) -> None:
        button.button_style = active_style if button.value else ''

    def make_style_updater(button, active_style: str):
        def updater(change):
            update_button_style(button, active_style)
        return updater

    for button in load_buttons.values():
        update_button_style(button, 'info')
        button.observe(make_style_updater(button, 'info'), names='value')

    for button in nic_buttons.values():
        update_button_style(button, 'success')
        button.observe(make_style_updater(button, 'success'), names='value')

    def set_all_buttons(buttons: Dict[str, ToggleButton], state: bool, style: str) -> None:
        for button in buttons.values():
            button.value = state
            update_button_style(button, style)

    activate_all_loads_button = Button(description="Activate All Loads", layout={'width': '160px'})
    deactivate_all_loads_button = Button(description="Deactivate All Loads", layout={'width': '160px'})
    activate_all_nics_button = Button(description="Activate All NICs", layout={'width': '160px'})
    deactivate_all_nics_button = Button(description="Deactivate All NICs", layout={'width': '160px'})

    activate_all_loads_button.on_click(lambda b: set_all_buttons(load_buttons, True, 'info'))
    deactivate_all_loads_button.on_click(lambda b: set_all_buttons(load_buttons, False, ''))
    activate_all_nics_button.on_click(lambda b: set_all_buttons(nic_buttons, True, 'success'))
    deactivate_all_nics_button.on_click(lambda b: set_all_buttons(nic_buttons, False, ''))

    core_slider = IntSlider(
        value=num_cores,
        min=1,
        max=num_cores,
        step=1,
        description="Cores:",
        continuous_update=False
    )

    selected_cores_label = Label(f"Selected Cores: {core_slider.value}")

    def update_selected_cores_label(change) -> None:
        selected_cores_label.value = f"Selected Cores: {change['new']}"

    core_slider.observe(update_selected_cores_label, names='value')

    method_toggle = ToggleButtons(
            options=[('Linear', 'linear'), ('Polynomial', 'polynomial')],
            value=method,
            description='Method:',
            style={'button_width': '120px'},
            layout={'width': '300px'}
        )

    plot_output = Output()

    # === Cache NIC power calculation ===
    nic_power_cache = {"active_nics": [], "total_power": 0.0}

    def plot(cores_selected: int, method: str, **kwargs) -> None:
        with plot_output:
            clear_output(wait=True)

            active_load_levels = [name for name, active in kwargs.items() if name in load_buttons and active]
            active_nics = [name for name, active in kwargs.items() if name in nic_buttons and active]

            if not active_load_levels:
                print("Please select at least one load level.")
                return

            # Cache total NIC power unless NIC selection changed
            if active_nics != nic_power_cache["active_nics"]:
                nic_power_cache["active_nics"] = active_nics
                nic_power_cache["total_power"] = sum(nic_power_lookup(model) for model in active_nics)

            total_nic_power = nic_power_cache["total_power"]

            labels = active_load_levels
            cpu_powers = []
            nic_powers = []

            for label in active_load_levels:
                load = load_options[label]
                active_cores = cores_selected * load
                if active_cores == 0:
                    predicted_cpu_power = cpu_model["linear_model"]["p_base"]
                else:
                    try:
                        predicted_cpu_power = predict_power(cpu_model, num_cores=active_cores, method=method)
                    except ValueError as e:
                        print(str(e))
                        return

                cpu_powers.append(predicted_cpu_power)
                nic_powers.append(total_nic_power)

            fig, ax = plt.subplots(figsize=(9, 6))
            for label, cpu, nic in zip(labels, cpu_powers, nic_powers):
                ax.bar(label, cpu, color=CPU_COLOR, edgecolor="black")
                ax.bar(label, nic, bottom=cpu, color=NIC_COLOR, edgecolor="black")
                yval = cpu + nic
                ax.text(label, yval + 1, f"{yval:.1f} W", ha='center', va='bottom', fontsize=10)

            ax.set_title(f"Predicted Server Power ({method.capitalize()} Model) - Node: {cpu_model.get('node_name', 'Unknown')}")
            ax.set_ylabel("Power (W)")
            cpu_patch = mpatches.Patch(color=CPU_COLOR, label='CPU Power')
            nic_patch = mpatches.Patch(color=NIC_COLOR, label='NIC Power')
            ax.legend(handles=[cpu_patch, nic_patch], loc='upper left')

            y_max = max([cpu + nic for cpu, nic in zip(cpu_powers, nic_powers)]) * 1.1
            ax.set_ylim(0, y_max)
            ax.grid(axis="y", linestyle="--", alpha=0.7)
            ax.tick_params(axis='x')
            fig.tight_layout()
            plt.show()

    ui = VBox([
        Label(value="------------------ CPU Load Configuration ------------------", layout={"font_weight": "bold", "margin": "10px 0 10px 0"}),
        VBox([
            HBox(list(load_buttons.values()), layout={'flex_wrap': 'wrap', 'gap': '6px'}),
            HBox([activate_all_loads_button, deactivate_all_loads_button], layout={'gap': '10px'})
        ]),
        Label(value="------------------ Active Network Interfaces (NICs) ------------------", layout={"font_weight": "bold", "margin": "10px 0 10px 0"}),
        VBox([
            HBox(list(nic_buttons.values()), layout={'flex_wrap': 'wrap', 'gap': '6px'}),
            HBox([activate_all_nics_button, deactivate_all_nics_button], layout={'gap': '10px'})
        ]),
        Label(value="------------------ CPU Core Configuration ------------------", layout={"font_weight": "bold", "margin": "10px 0 10px 0"}),
        selected_cores_label,
        core_slider,
        Label(value="------------------ Prediction Model ------------------", layout={"font_weight": "bold", "margin": "10px 0 10px 0"}),
        method_toggle
    ])

    control_dict = {**load_buttons, **nic_buttons, 'cores_selected': core_slider, 'method': method_toggle}
    _ = interactive_output(plot, control_dict)

    display(VBox([ui, plot_output]))

def load_cpu_model() -> Dict:
    server_name = input("Server Name: ")
    name = f"{MODEL_OUTPUT}_{server_name}.json"
    path = os.path.join(CPU_MODEL_FOLDER, name)

    if not os.path.exists(path):
        raise FileNotFoundError(f"Could not find cpu_model file {path}")
    with open(path, "r") as file:
        model = json.load(file)
        print(f"Loaded model from {path}")
        return model

def load_all_cpu_models() -> Dict[str, Dict]:
    models = {}
    for filename in os.listdir(CPU_MODEL_FOLDER):
        if filename.startswith(MODEL_OUTPUT) and filename.endswith(".json"):
            path = os.path.join(CPU_MODEL_FOLDER, filename)
            with open(path, "r") as file:
                model = json.load(file)
                node_name = model.get("node_name", filename)
                models[node_name] = model
    if not models:
        raise FileNotFoundError(f"No CPU model files found in {CPU_MODEL_FOLDER}.")
    print(f"Loaded {len(models)} models.")
    return models

def predict_power(model: Dict, num_cores: int, method: str = "linear") -> float:
    if method == "linear":
        if model["model_type"] == "linear_fixed_idle":
            p_core = model["linear_model"]["p_core"]
            p_base = model["linear_model"]["p_base"]
            return p_core * num_cores + p_base
        elif model["model_type"] == "linear_free_fit":
            p_core = model["linear_model"]["p_core"]
            intercept = model["linear_model"]["fitted_intercept"]
            return p_core * num_cores + intercept
        else:
            raise ValueError(f"Unknown linear model type: {model['model_type']}")
    elif method == "polynomial":
        if "polynomial_model" not in model:
            raise ValueError("Polynomial model coefficients not available in model.")
        a = model["polynomial_model"]["a"]
        b = model["polynomial_model"]["b"]
        c = model["polynomial_model"]["c"]
        return a * (num_cores ** 2) + b * num_cores + c
    else:
        raise ValueError(f"Unknown prediction method: {method}")

def select_and_plot_cpu_model(models: Dict[str, Dict], method: str = "linear") -> None:
    dropdown = Dropdown(
        options=list(models.keys()),
        value=None,
        description="Node:",
        layout={'width': '300px'}
    )

    output = Output()

    def on_select(change):
        if change['type'] == 'change' and change['name'] == 'value' and change['new'] is not None:
            with output:
                clear_output(wait=True)
                selected_node = change['new']
                interactive_prediction_plots(cpu_model=models[selected_node], method=method)

    dropdown.observe(on_select, names='value')

    display(VBox([
        Label("Select Node to Visualize:"),
        dropdown,
        output
    ]))


# All Models

In [ ]:
models = load_all_cpu_models()
select_and_plot_cpu_model(models)

# Combined

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import os
import json
from ipywidgets import ToggleButton, VBox, HBox, IntSlider, Label, Output, ToggleButtons, Button
from IPython.display import display, clear_output
from typing import Dict

# === Assume these are defined ===
# NIC_POWER_DATABASE = {...}
# CPU_COLOR = "#17a2b8"
# NIC_COLOR = "#28a745"

def nic_power_lookup(model_name: str) -> float:
    return NIC_POWER_DATABASE.get(model_name, 0.0)

def adjust_color(base: str, factor: float) -> str:
    rgb = mcolors.to_rgb(base)
    adjusted = tuple(min(1, c * factor) for c in rgb)
    return mcolors.to_hex(adjusted)

def interactive_multi_node(models: Dict[str, Dict]) -> None:
    # Global Section
    load_options = ["Idle (0%)", "25% Load", "50% Load", "75% Load", "100% Load"]

    global_load_buttons = {}

    def update_global_load_button_style(btn):
        btn.button_style = 'info' if btn.value else ''

    for name in load_options:
        btn = ToggleButton(value=True, description=name, layout={'width': '120px'})
        update_global_load_button_style(btn)
        btn.observe(lambda change, b=btn: (update_global_load_button_style(b), plot()), names='value')
        global_load_buttons[name] = btn

    method_toggle = ToggleButtons(
        options=[('Linear', 'linear'), ('Polynomial', 'polynomial')],
        value='linear'
    )

    def toggle_all(buttons, value, style_func):
        for b in buttons.values():
            b.value = value
            style_func(b)

    activate_loads = Button(description="Activate All Loads", layout={'width': '160px'})
    deactivate_loads = Button(description="Deactivate All Loads", layout={'width': '160px'})
    activate_loads.on_click(lambda _: toggle_all(global_load_buttons, True, update_global_load_button_style))
    deactivate_loads.on_click(lambda _: toggle_all(global_load_buttons, False, update_global_load_button_style))

    # Node Selection
    node_buttons = {
        node: ToggleButton(value=False, description=node, layout={'width': '120px'})
        for node in models
    }

    node_settings = {}
    node_configs_output = Output()
    plot_output = Output()

    def build_node_config(node_name: str):
        model = models[node_name]
        num_cores = model.get("cpu_info", {}).get("cores", 4)

        nic_buttons = {
            nic: ToggleButton(value=True, description=nic, layout={'width': '140px'})
            for nic in NIC_POWER_DATABASE
        }

        def on_nic_change(change, b):
            b.button_style = 'success' if b.value else ''
            plot()

        for b in nic_buttons.values():
            b.button_style = 'success' if b.value else ''
            b.observe(lambda change, b=b: on_nic_change(change, b), names='value')


        core_slider = IntSlider(value=num_cores, min=1, max=num_cores, step=1, description="Cores:")
        core_slider.observe(lambda c: plot(), names='value')

        activate_nics = Button(description="Activate All NICs", layout={'width': '120px'})
        deactivate_nics = Button(description="Deactivate All NICs", layout={'width': '120px'})
        activate_nics.on_click(lambda _: toggle_all(nic_buttons, True, lambda b: setattr(b, 'button_style', 'success')))
        deactivate_nics.on_click(lambda _: toggle_all(nic_buttons, False, lambda b: setattr(b, 'button_style', '')))

        node_settings[node_name] = {
            'model': model,
            'nic_buttons': nic_buttons,
            'core_slider': core_slider
        }

        capitalized = node_name.capitalize()
        return VBox([
            Label(f"=============== {capitalized} Configuration ===============", style={'font_weight': 'bold', 'font_size': '16px'}),
            VBox([
                Label("Select NICs", style={'font_weight': 'bold'}),
                HBox(list(nic_buttons.values()), layout={'flex_wrap': 'wrap'}),
                HBox([activate_nics, deactivate_nics], layout={'gap': '8px'})
            ]),
            Label("Select Number of Cores", style={'font_weight': 'bold'}),
            core_slider
        ], layout={'margin': '25px 0 0 0'})

    def update_node_button_style(btn):
        btn.button_style = 'warning' if btn.value else ''

    def update_node_configs(change=None):
        with node_configs_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            display(VBox([build_node_config(n) for n in active_nodes]))
            plot()

    for node, btn in node_buttons.items():
        update_node_button_style(btn)
        btn.observe(lambda change, b=btn: (update_node_button_style(b), update_node_configs()), names='value')

    method_toggle.observe(lambda c: plot(), names='value')

    def plot():
        with plot_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            selected_loads = [name for name, b in global_load_buttons.items() if b.value]

            if not active_nodes:
                print("Select at least one node.")
                return
            if not selected_loads:
                print("Select at least one load level.")
                return

            for idx, node in enumerate(active_nodes):
                s = node_settings[node]
                model = s['model']
                cores = s['core_slider']
                nics = s['nic_buttons']
                method = method_toggle.value

                nic_total = sum(nic_power_lookup(n) for n, b in nics.items() if b.value)

                fig, ax = plt.subplots(figsize=(8, 5))
                x = range(len(selected_loads))
                cpu_powers = []
                nic_powers = []

                for load_label in selected_loads:
                    load_map = {
                        "Idle (0%)": 0.0,
                        "25% Load": 0.25,
                        "50% Load": 0.5,
                        "75% Load": 0.75,
                        "100% Load": 1.0
                    }
                    load = load_map[load_label]
                    active_cores = cores.value * load
                    if active_cores == 0:
                        cpu_power = model["linear_model"]["p_base"]
                    else:
                        cpu_power = predict_power(model, active_cores, method)

                    cpu_powers.append(cpu_power)
                    nic_powers.append(nic_total)

                factor = 1 - idx * 0.1
                cpu_color = adjust_color(CPU_COLOR, factor)
                nic_color = adjust_color(NIC_COLOR, factor)

                bottoms = [0] * len(selected_loads)
                ax.bar(x, cpu_powers, bottom=bottoms, color=cpu_color, edgecolor='black')
                bottoms = [c for c in cpu_powers]
                ax.bar(x, nic_powers, bottom=bottoms, color=nic_color, edgecolor='black')

                for i, total in enumerate([c+n for c, n in zip(cpu_powers, nic_powers)]):
                    ax.text(x[i], total + 1, f"{total:.1f} W", ha='center', fontsize=10)

                ax.set_xticks(list(x))
                ax.set_xticklabels(selected_loads, rotation=15)
                ax.set_ylim(0, max([c+n for c, n in zip(cpu_powers, nic_powers)]) * 1.2)
                ax.set_ylabel("Power (W)")
                ax.set_title(f"Power Prediction for {node.capitalize()}")
                ax.grid(axis="y", linestyle="--", alpha=0.6)
                cpu_patch = mpatches.Patch(color=cpu_color, label='CPU Power')
                nic_patch = mpatches.Patch(color=nic_color, label='NIC Power')
                ax.legend(handles=[cpu_patch, nic_patch], loc='upper left')
                fig.tight_layout()
                plt.show()

    # === Initial Display ===
    display(VBox([
        Label("========= Global Settings =========", style={'font_weight': 'bold', 'font_size': '20px'}),
        VBox([
            Label("Select Load Levels (Global)", style={'font_weight': 'bold'}),
            HBox(list(global_load_buttons.values()), layout={'flex_wrap': 'wrap'}),
            HBox([activate_loads, deactivate_loads], layout={'gap': '8px'}),
            Label("Select Prediction Method", style={'font_weight': 'bold'}),
            method_toggle
        ]),
        Label("========= Node Selection =========", style={'font_weight': 'bold', 'font_size': '20px'}),
        HBox(list(node_buttons.values())),
        node_configs_output,
        plot_output
    ]))


In [ ]:
interactive_multi_node(models)

# New

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import os
import json
from ipywidgets import ToggleButton, VBox, HBox, IntSlider, Label, Output, ToggleButtons, Button
from IPython.display import display, clear_output
from typing import Optional, Dict

# Assume NIC_POWER_DATABASE and CPU_COLOR/NIC_COLOR are already defined

def nic_power_lookup(model_name: str) -> float:
    return NIC_POWER_DATABASE.get(model_name, 0.0)

def adjust_color(base: str, factor: float) -> str:
    rgb = mcolors.to_rgb(base)
    adjusted = tuple(min(1, c * factor) for c in rgb)
    return mcolors.to_hex(adjusted)

def interactive_multi_node(models: Dict[str, Dict]) -> None:
    node_buttons = {
        node: ToggleButton(value=False, description=node, layout={'width': '120px'}, button_style='')
        for node in models
    }

    node_settings = {}
    node_configs_output = Output()
    plot_output = Output()

    def build_node_config(node_name: str):
        model = models[node_name]
        num_cores = model.get("cpu_info", {}).get("cores", 4)

        load_options = {
            "Idle (0%)": 0.0,
            "25% Load": 0.25,
            "50% Load": 0.5,
            "75% Load": 0.75,
            "100% Load": 1.0
        }

        # Create load buttons
        load_buttons = {}

        def update_style(btn, active_style):
            btn.button_style = active_style if btn.value else ''

        def on_load_change(change, name):
            if change['new']:
                for other_name, other_btn in load_buttons.items():
                    if other_name != name:
                        other_btn.unobserve_all()
                        other_btn.value = False
                        update_style(other_btn, '')  # Make grey
                        other_btn.observe(lambda c, n=other_name: on_load_change(c, n), names='value')
                    else:
                        update_style(other_btn, 'info')  # Blue for active
                plot()

        for name in load_options:
            btn = ToggleButton(value=False, description=name, layout={'width': '120px'})
            btn.observe(lambda change, n=name: on_load_change(change, n), names='value')
            load_buttons[name] = btn

        # Optional: default selection = Idle (0%)
        default = "Idle (0%)"
        load_buttons[default].unobserve_all()
        load_buttons[default].value = True
        update_style(load_buttons[default], 'info')
        load_buttons[default].observe(lambda c, n=default: on_load_change(c, n), names='value')




        nic_buttons = {
            nic: ToggleButton(value=True, description=nic, layout={'width': '140px'})
            for nic in NIC_POWER_DATABASE
        }

        def update_style(btn, active_style):
            btn.button_style = active_style if btn.value else ''

        def observe_button(button, active_style):
            def change(change):
                update_style(button, active_style)
                plot()
            button.observe(change, names='value')

        for b in nic_buttons.values():
            update_style(b, 'success')
            observe_button(b, 'success')

        core_slider = IntSlider(value=num_cores, min=1, max=num_cores, step=1, description="Cores:")
        core_slider.observe(lambda c: plot(), names='value')

        method_toggle = ToggleButtons(
            options=[('Linear', 'linear'), ('Polynomial', 'polynomial')],
            value='linear'
        )
        method_toggle.observe(lambda c: plot(), names='value')

        # Activate/Deactivate NICs
        def toggle_all(buttons, value, style):
            for b in buttons.values():
                b.value = value
                update_style(b, style)

        activate_nics = Button(description="Activate All", layout={'width': '120px'})
        deactivate_nics = Button(description="Deactivate All", layout={'width': '120px'})
        activate_nics.on_click(lambda _: toggle_all(nic_buttons, True, 'success'))
        deactivate_nics.on_click(lambda _: toggle_all(nic_buttons, False, ''))

        node_settings[node_name] = {
            'model': model,
            'load_buttons': load_buttons,
            'nic_buttons': nic_buttons,
            'core_slider': core_slider,
            'method_toggle': method_toggle,
            'load_options': load_options
        }

        capitalized = node_name.capitalize()
        return VBox([
            Label(f"=============== {capitalized} Configuration ===============", style={'font_weight': 'bold', 'font_size': '16px'}),
            VBox([
                Label("Select CPU Load Level", style={'font_weight': 'bold'}),
                HBox(list(load_buttons.values()), layout={'flex_wrap': 'wrap'})
            ]),
            VBox([
                Label("Select NICs", style={'font_weight': 'bold'}),
                HBox(list(nic_buttons.values()), layout={'flex_wrap': 'wrap'}),
                HBox([activate_nics, deactivate_nics], layout={'gap': '8px'})
            ]),
            Label("Select Prediction Method", style={'font_weight': 'bold'}),
            method_toggle,
            core_slider
        ], layout={'margin': '25px 0 0 0'})

    def update_node_configs(change=None):
        with node_configs_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            display(VBox([build_node_config(n) for n in active_nodes]))
            plot()

    def update_node_button_style(btn):
        btn.button_style = 'warning' if btn.value else ''

    for node, btn in node_buttons.items():
        update_node_button_style(btn)
        btn.observe(lambda change, b=btn: (update_node_button_style(b), update_node_configs()), names='value')



    def plot():
        with plot_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            if not active_nodes:
                print("Select at least one node.")
                return

            fig, ax = plt.subplots(figsize=(6, 6))
            bottoms = [0]  # Only one bar (x=0)
            legend_handles = []

            for idx, node in enumerate(active_nodes):
                s = node_settings[node]
                model = s['model']
                cores = s['core_slider']
                method = s['method_toggle'].value
                loads = s['load_buttons']
                nics = s['nic_buttons']
                opt = s['load_options']

                # Find selected load
                selected_load = None
                for name, btn in loads.items():
                    if btn.value:
                        selected_load = name
                        break

                if selected_load is None:
                    continue  # skip node if no load selected

                load_value = opt[selected_load]
                nic_total = sum(nic_power_lookup(n) for n, b in nics.items() if b.value)
                factor = 1 - idx * 0.1
                c_color = adjust_color(CPU_COLOR, factor)
                n_color = adjust_color(NIC_COLOR, factor)

                legend_handles.append(mpatches.Patch(color=c_color, label=f"CPU ({node})"))
                legend_handles.append(mpatches.Patch(color=n_color, label=f"NIC ({node})"))

                active_cores = cores.value * load_value
                cpu = model["linear_model"]["p_base"] if active_cores == 0 else predict_power(model, active_cores, method)

                cpu_height = cpu
                nic_height = nic_total

                ax.bar(0, cpu_height, bottom=bottoms[0], color=c_color, edgecolor='black')
                bottoms[0] += cpu_height

                ax.bar(0, nic_height, bottom=bottoms[0], color=n_color, edgecolor='black')
                bottoms[0] += nic_height

            # Watt label
            total = bottoms[0]
            ax.text(0, total + 3, f"{total:.1f} W", ha='center', fontsize=12)

            ax.set_xlim(-0.5, 0.5)
            ax.set_xticks([0])
            ax.set_xticklabels(["System Configuration"], rotation=0)
            ax.set_ylim(0, total * 1.2)
            ax.set_ylabel("Power (W)")
            ax.set_title("Total System Power Prediction")
            ax.grid(axis="y", linestyle="--", alpha=0.6)
            ax.legend(handles=legend_handles, loc='upper left')
            fig.tight_layout()
            plt.show()

    # Initial render
    update_node_configs()
    display(VBox([
        Label("Select Nodes:", style={'font_weight': 'bold', 'font_size': '20px'}),
        HBox(list(node_buttons.values())),
        node_configs_output,
        plot_output
    ]))


In [ ]:
interactive_multi_node(models)